# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [14]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [15]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'
i = 0
# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
file_path_list

/home/workspace


['/home/workspace/event_data/2018-11-27-events.csv',
 '/home/workspace/event_data/2018-11-04-events.csv',
 '/home/workspace/event_data/2018-11-07-events.csv',
 '/home/workspace/event_data/2018-11-09-events.csv',
 '/home/workspace/event_data/2018-11-19-events.csv',
 '/home/workspace/event_data/2018-11-05-events.csv',
 '/home/workspace/event_data/2018-11-22-events.csv',
 '/home/workspace/event_data/2018-11-16-events.csv',
 '/home/workspace/event_data/2018-11-26-events.csv',
 '/home/workspace/event_data/2018-11-24-events.csv',
 '/home/workspace/event_data/2018-11-29-events.csv',
 '/home/workspace/event_data/2018-11-15-events.csv',
 '/home/workspace/event_data/2018-11-20-events.csv',
 '/home/workspace/event_data/2018-11-06-events.csv',
 '/home/workspace/event_data/2018-11-18-events.csv',
 '/home/workspace/event_data/2018-11-21-events.csv',
 '/home/workspace/event_data/2018-11-10-events.csv',
 '/home/workspace/event_data/2018-11-23-events.csv',
 '/home/workspace/event_data/2018-11-02-events

#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [16]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader) # return the next row, 즉 head skip하기 

 # extracting each data row one by one and append it        
        for line in csvreader:
            full_data_rows_list.append(line) 

# uncomment the code below if you would like to get total number of rows 
#print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
#csv.QUOTE_ALL은 '' 안에 들어있는 모든걸 하나로 묶겠다는 의미
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        # cassandra에 들어갈 column select
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [17]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [18]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [19]:
# TO-DO: Create a Keyspace 
session.execute("""CREATE KEYSPACE IF NOT EXISTS udacity 
                    WITH REPLICATION={
                    'class' : 'SimpleStrategy',
                    'replication_factor': 1
                    }""")

#### Set Keyspace

In [20]:
# TO-DO: Set KEYSPACE to the keyspace specified above
session.execute("""USE udacity""")

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




In [26]:
## TO-DO: Query 1:  Give me the artist, song title and song's length in the music app history that was heard during \
## sessionId = 338, and itemInSession = 4
#session.execute("DROP TABLE udacity.music_library")
session.execute("""CREATE TABLE IF NOT EXISTS udacity.music_library(
                    artist text,
                    itemInSession int,
                    length decimal,
                    sessionId int,
                    song text,
                    PRIMARY KEY (sessionId, itemInSession)
                    )
                """)

                    

In [29]:
# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below#
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## TO-DO: Assign the INSERT statements into the `query` variable
        query = """INSERT INTO udacity.music_library (artist, itemInSession, length, sessionId, song)"""
        query = query + "VALUES(%s, %s, %s, %s, %s)"
        ## TO-DO: Assign which column element should be assigned for each column in the INSERT statement.
        ## For e.g., to INSERT artist_name and user first_name, you would change the code below to `line[0], line[1]`
        session.execute(query, (line[0], int(line[3]), float(line[5]), int(line[8]), line[9]))

#### Do a SELECT to verify that the data have been inserted into each table

In [38]:
## TO-DO: Add in the SELECT statement to verify the data was entered into the table
rows = session.execute("SELECT artist, song, length FROM music_library WHERE sessionId = 338 AND itemInSession = 4")
for row in rows:
    print(row.artist, ' ', row.song, ' ', row.length, ' ')


Faithless   Music Matters (Mark Knight Dub)   495.3073  


### COPY AND REPEAT THE ABOVE THREE CELLS FOR EACH OF THE THREE QUESTIONS

In [50]:
session.execute("""DROP TABLE music_library2""")
## TO-DO: Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name)\
## for userid = 10, sessionid = 182
session.execute("""CREATE TABLE IF NOT EXISTS udacity.music_library2(
                    artist text,
                    song text,
                    sessionId int,
                    userId int,
                    firstName text,
                    lastName text,
                    itemInSession int,
                    PRIMARY KEY (userId, sessionId, itemInSession)
                    )
                """)

Row(artist='Down To The Bone', song="Keep On Keepin' On", firstname='Sylvie', lastname='Cruz')
Row(artist='Three Drives', song='Greece 2000', firstname='Sylvie', lastname='Cruz')
Row(artist='Sebastien Tellier', song='Kilometer', firstname='Sylvie', lastname='Cruz')
Row(artist='Lonnie Gordon', song='Catch You Baby (Steve Pitron & Max Sanna Radio Edit)', firstname='Sylvie', lastname='Cruz')


In [ ]:
# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below#
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## TO-DO: Assign the INSERT statements into the `query` variable
        query = """INSERT INTO udacity.music_library2 (artist, song, sessionId, userId, firstName, lastName, itemInSession)"""
        query = query + "VALUES(%s, %s, %s, %s, %s, %s, %s)"
        ## TO-DO: Assign which column element should be assigned for each column in the INSERT statement.
        ## For e.g., to INSERT artist_name and user first_name, you would change the code below to `line[0], line[1]`
        session.execute(query, (line[0], line[9], int(line[8]), int(line[10]), line[1], line[4], int(line[3])))

In [ ]:
## TO-DO: Add in the SELECT statement to verify the data was entered into the table
rows = session.execute("SELECT artist, song, ifirstName, lastName FROM music_library2 WHERE userId = 10 AND sessionId = 182")
for row in rows:
    print(row)

In [55]:
## TO-DO: Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'
session.execute("""CREATE TABLE IF NOT EXISTS udacity.music_library3(
                    firstName text,
                    lastName text,
                    userId int,
                    song text,
                    sessionId int,
                    itemInSession int,
                    PRIMARY KEY (song, sessionId, itemInSession)
                    )
                """)

                    

In [59]:
# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below#
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## TO-DO: Assign the INSERT statements into the `query` variable
        query = """INSERT INTO udacity.music_library23 (firstName, lastName, userId, song, sessionId, itemInSession)"""
        query = query + "VALUES(%s, %s, %s, %s, %s, %s)"
        ## TO-DO: Assign which column element should be assigned for each column in the INSERT statement.
        ## For e.g., to INSERT artist_name and user first_name, you would change the code below to `line[0], line[1]`
        session.execute(query, (line[1], line[4], int(line[10]), line[9], int(line[8]), int(line[3])))

In [ ]:
## TO-DO: Add in the SELECT statement to verify the data was entered into the table
rows = session.execute("SELECT firstName, lastName FROM music_librar32 WHERE userId = 10 AND sessionId = 182")
for row in rows:
    print(row)

### Drop the tables before closing out the sessions

In [63]:
## TO-DO: Drop the table before closing out the sessions
session.execute("""DROP TABLE music_library""")
session.execute("""DROP TABLE music_library2""")
session.execute("""DROP TABLE music_library3""")

### Close the session and cluster connection¶

In [64]:
session.shutdown()
cluster.shutdown()